In [ ]:
#インストール関連
!pip install basicsr
%cd /notebooks 
!git clone https://github.com/s0md3v/roop.git
%cd /notebooks/roop
!pip install onnxruntime-gpu && pip install -r requirements.txt
!wget https://huggingface.co/MonsterMMORPG/SECourses/resolve/main/inswapper_128.onnx
%cd /notebooks
!rm -rf CodeFormer
!git clone https://github.com/s3xmaxman/CodeFormer.git

%cd CodeFormer
!pip install -r requirements.txt
!python basicsr/setup.py develop

!python scripts/download_pretrained_models.py facelib
!python scripts/download_pretrained_models.py CodeFormer

from function import *

! mkdir download
%cd /notebooks/roop
!sudo apt-get -y install python3.9-tk
!pip install torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cu118
clear_output()
print("Done!!!!!")

In [ ]:
#容量が大きいファイルはGoogle driveの共有リンクから持ってきたほうが早いのでその場合に使用してください
#詳しくは下記のURLから
#https://docs.paperspace.com/gradient/notebooks/notebook-storage/
!gdown "your_link_id" -O /notebooks/roop/video.mp4

In [ ]:
#Face Swap!!
%cd /notebooks/roop
face_image = "face.png"  
video_file = "video.mp4"  
output_vid = "output.mp4"  
swap_all_faces = False  
keep_fps = True  
gpu_threads = 8  

keep_fps_opt = "--keep-fps" if keep_fps else ""

!python run.py -f $face_image -t $video_file -o $output_vid --gpu-vendor nvidia --gpu-threads $gpu_threads $keep_fps_opt

In [ ]:
#動画の読み込み
!cp /notebooks/roop/output.mp4 /notebooks/CodeFormer/movie
%cd /notebooks/CodeFormer
video = 'output.mp4'
video_file = '/notebooks/CodeFormer/movie/'+video
image_dir='/notebooks/CodeFormer/inputs/frame/'
image_file='%s.png'

# video_2_images
reset_folder('/notebooks/CodeFormer/inputs/frame')
fps, i, interval = video_2_images(video_file, image_dir, image_file)

#スタートフレーム表示
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('/notebooks/CodeFormer/inputs/frame/000000.png')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()


# パラメータ表示
print('fps = ', fps)
print('frames = ', i)
print('interval = ', interval)

In [ ]:
#CoderFormer
from types import FrameType

# 設定
input_folder = 'inputs/frame'
w = 0.85
upscaled_flame = reset_folder('results/frame_'+str(w))

#フレームの高画質化
! python inference_codeformer.py --w $w\
                                   --test_path $input_folder\
                                   --bg_upsampler realesrgan\
                                    --face_upsample
# ログ・クリア
clear_output()

# 動画の作成
print('makeing movie...')
fps_r = fps/interval
file_path = 'results/frame_'+str(w)+'/final_results/%06d.png'
! ffmpeg -y -r $fps_r -i $file_path -vcodec libx264 -pix_fmt yuv420p -loglevel error out.mp4

# 音声の抽出・付加
print('preparation for sound...')
! ffmpeg -y -i $video_file -loglevel error sound.mp3
! ffmpeg -y -i out.mp4 -i sound.mp3 -loglevel error output.mp4

# 動画の再生
print('waiting for play movie...')
display_mp4('output.mp4')

In [ ]:
#pydrive関連のインストール
!pip install pydrive
!pip install -U httplib2==0.15.0

In [ ]:
#大きなoutput.mp4はダウンロードに時間が掛かるのでgoogle driveにアップロードする
#すこし作業が必要になるので下記のサイトをみてください
#https://onl.tw/8VFSknG
!cp /notebooks/CodeFormer/output.mp4 /notebooks/output.mp4
%cd /notebooks
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.service_account import ServiceAccountCredentials
 
JSON_FILE = "service_account.json"
ID = "your_service_account_id"
 
gauth = GoogleAuth()
scope = ["https://www.googleapis.com/auth/drive"]
gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name(JSON_FILE, scope)
drive = GoogleDrive(gauth)
 
# ファイルをGoogle Driveにアップロード
f = drive.CreateFile({"parents": [{"id": ID }]})
f.SetContentFile('output.mp4')
f.Upload()

print('ファイルがアップロードされました。')

In [ ]:
#ディレクトリの削除
!rm -r /notebooks/roop
!rm -r /notebooks/CodeFormer